In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
def load_data(data_path):
    emotions = []
    features = []

    for dirpath, dirnames, filenames in os.walk(data_path):
        for file in filenames:
            if file.endswith(".wav"):
                emotion = file.split('-')[2]
                if emotion in ['03', '04', '05']:  # Assuming 03 = happy, 04 = sad, 05 = anger
                    emotions.append(emotion)
                    file_path = os.path.join(dirpath, file)
                    X, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
                    mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40)
                    mfccs_scaled = np.mean(mfccs.T, axis=0)
                    features.append(mfccs_scaled)

    return np.array(features), np.array(emotions)

data_path = "put/your/path"
X, y = load_data(data_path)


In [ ]:
label_mapping = {'03': 'happy', '04': 'sad', '05': 'anger'}
y_mapped = np.array([label_mapping[emotion] for emotion in y])

lb = LabelEncoder()
y_encoded = lb.fit_transform(y_mapped)
y = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensure that y_test_encoded is available for later use in evaluation
y_test_encoded = y_encoded[y_test.argmax(axis=1)]


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
model = Sequential()
model.add(Conv1D(256, 5, padding='same', input_shape=(X_train.shape[1], 1), activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Dropout(0.5))

model.add(Conv1D(128, 5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Reshape data for Conv1D
X_train_reshaped = np.expand_dims(X_train, axis=2)
X_test_reshaped = np.expand_dims(X_test, axis=2)

history = model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, validation_data=(X_test_reshaped, y_test), verbose=1)


In [ ]:
y_pred = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred, axis=1)

print("Accuracy:", accuracy_score(y_test_encoded, y_pred))

# Create string labels for the emotions
emotion_labels = lb.classes_

print(classification_report(y_test_encoded, y_pred, target_names=emotion_labels))


In [ ]:
def plot_history(history):
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.show()

plot_history(history)


In [ ]:
cm = confusion_matrix(y_test_encoded, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=emotion_labels, yticklabels=emotion_labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()
